In [ ]:
import fastf1
from matplotlib import pyplot as plt
import fastf1
import fastf1.plotting as f1_plt
import pandas as pd
import utils as ut

In [ ]:
# gettting the data
session = ut.get_session()

In [ ]:
q1, q2, q3 = ut.get_qualifying(session)
fast_mag = q3.pick_driver('MAG')
fast_ver = q3.pick_driver('VER')

In [ ]:
best_lap_mag = fast_mag.pick_lap(25).get_car_data()
best_lap_tel_mag = fast_mag.pick_lap(25).get_telemetry()

best_lap_ver = fast_ver.pick_lap(22).get_car_data()
best_lap_tel_ver = fast_ver.pick_lap(22).get_telemetry()

In [ ]:
x_mag = best_lap_tel_mag["X"]
y_mag = best_lap_tel_mag["Y"]
x_ver = best_lap_tel_ver["X"]
y_ver = best_lap_tel_ver["Y"]

In [ ]:
best_lap_tel_mag["DistanceToDriverAhead"] = 1
best_lap_tel_mag 

In [ ]:
best_lap_tel_ver

In [ ]:
best_lap_tel_ver.where(best_lap_tel_ver["Source"] == "pos").dropna()

In [ ]:
ani_mag = best_lap_tel_mag.where((best_lap_tel_mag["Source"] == "pos") | (best_lap_tel_mag["Source"] == "interpolation")).dropna()
ani_ver = best_lap_tel_ver.where((best_lap_tel_ver["Source"] == "pos") | (best_lap_tel_ver["Source"] == "interpolation")).dropna()

ani_speed_mag = [speed for speed in ani_mag["Speed"]]
ani_speed_ver = [speed for speed in ani_ver["Speed"]]

ani_x_mag = ani_mag["X"]
ani_x_ver = ani_ver["X"]

ani_y_mag = ani_mag["Y"]
ani_y_ver = ani_ver["Y"]

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation


def update_plot(i, scat_mag, scat_ver, line_mag, line_ver):
    
    line_mag.set_data(range(i), ani_speed_mag[:i])
    line_ver.set_data(range(i), ani_speed_ver[:i])


    index = ani_mag.index[i]
    plot_x_mag = ani_x_mag[index]
    plot_y_mag = ani_y_mag[index]
    scat_mag.set_offsets((plot_y_mag, -plot_x_mag))



    index = ani_ver.index[i]
    plot_x_ver = ani_x_ver[index]
    plot_y_ver = ani_y_ver[index]
    scat_ver.set_offsets((plot_y_ver, -plot_x_ver))
    
    return scat_mag, scat_ver, line_mag, line_ver

numframes = len(ani_mag)

index = ani_mag.index[0]

plot_x_mag = ani_x_mag[index]
plot_y_mag = ani_y_mag[index]


index = ani_ver.index[0]

plot_x_ver = ani_x_ver[index]
plot_y_ver = ani_y_ver[index]

data = range(numframes)

line_index = ani_mag.index[numframes-1]
line_x = ani_x_mag[index]
line_y = ani_y_mag[index]

fig, ax = plt.subplots(2,1, figsize=(16,9), gridspec_kw={'height_ratios': [2, 1]})

ax[0].plot(y_mag, -x_mag, label='MAG', c="black", linewidth=12, alpha=0.5, zorder = 0)
ax[0].plot([line_y, line_y- 350], [-line_x - 500, -line_x + 500], c = "black", linewidth=3)

speed_line_mag = ax[1].plot(range(0), ani_speed_mag[:0], c="black")
speed_line_ver = ax[1].plot(range(0), ani_speed_ver[:0], c="blue")


scat_mag = ax[0].scatter(plot_y_mag, -plot_x_mag, c="black", s=50)
scat_ver = ax[0].scatter(plot_y_ver, -plot_x_ver, c="blue", s=50)


ax[0].legend([scat_mag, scat_ver], ["Magnussen", "Verstappen"])
ax[0].set_xlim([-6000, 5500])
ax[0].set_ylim([-2500, 5500])
ax[0].xaxis.set_visible(False)
ax[0].yaxis.set_visible(False)

ax[1].set_xlim([0, numframes])
ax[1].set_ylim([0, max(ani_speed_mag)+ 10])
ax[1].xaxis.set_visible(False)



ani = animation.FuncAnimation(fig, update_plot, frames=range(numframes),
                                fargs=[scat_mag, scat_ver, speed_line_mag[0], speed_line_ver[0]], blit = False, interval=500, repeat=True)
plt.show()
# To save the animation using Pillow as a gif
writer = animation.PillowWriter(fps=30,
                                metadata=dict(artist='Sebastian'),
                                bitrate=1800)
ani.save('./figures/lap_animation.gif', writer=writer)

# to save the animation using FFMpegWritter as a mp4
writervideo = animation.FFMpegWriter(fps=30) # brew install ffmpeg (if not working)
ani.save('./figures/lap_animation.mp4', writer=writervideo)


